# Strands Agents를 사용한 도구로서의 에이전트


"도구로서의 에이전트(Agents as Tools)"는 AI 시스템의 아키텍처 패턴으로, 전문화된 AI 에이전트를 다른 에이전트가 사용할 수 있는 호출 가능한 함수(도구)로 래핑하는 것입니다. 이는 다음과 같은 계층적 구조를 만듭니다:

1. 주요 "오케스트레이터" 에이전트가 사용자 상호작용을 처리하고 어떤 전문 에이전트를 호출할지 결정합니다

2. 전문화된 "도구 에이전트"들이 오케스트레이터에 의해 호출될 때 도메인별 작업을 수행합니다

이 접근 방식은 관리자가 전문가들을 조정하는 인간 팀 역학을 모방하며, 각각이 복잡한 문제를 해결하기 위해 고유한 전문 지식을 제공합니다. 단일 에이전트가 모든 것을 처리하려고 하는 대신, 작업이 가장 적절한 전문 에이전트에게 위임됩니다.



## 주요 이점 및 핵심 원칙

"도구로서의 에이전트" 패턴은 여러 가지 장점을 제공합니다:

- 관심사의 분리: 각 에이전트는 집중된 책임 영역을 가지므로 시스템을 이해하고 유지 관리하기가 더 쉽습니다
- 계층적 위임: 오케스트레이터가 어떤 전문가를 호출할지 결정하여 명확한 지휘 체계를 만듭니다
- 모듈식 아키텍처: 전문가들을 전체 시스템에 영향을 주지 않고 독립적으로 추가, 제거 또는 수정할 수 있습니다
- 향상된 성능: 각 에이전트는 특정 작업에 최적화된 맞춤형 시스템 프롬프트와 도구를 가질 수 있습니다


In [ ]:
%pip install -r requirements.txt

In [ ]:
import os

from strands import Agent, tool
from strands_tools import file_write


이 모듈에서는 오케스트레이터 기반 멀티 에이전트 워크플로를 만들어보겠습니다.

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

또한 중첩된 에이전트를 만들 수 있게 해주는 `use_llm`도 살펴보겠습니다.

## 연구 에이전트

먼저 http_request 도구를 사용하는 기본 연구 어시스턴트를 만들어보겠습니다.

In [ ]:
RESEARCH_ASSISTANT_PROMPT = """당신은 전문 연구 어시스턴트입니다. 연구 질문에 대한 응답으로
사실적이고 잘 출처가 명시된 정보만 제공하는 데 집중하세요.
가능한 경우 항상 출처를 인용하세요."""

In [ ]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # 여기서 에이전트 AI 검색 도구를 활성화할 수 있습니다
)

query = "Amazon Bedrock과 그 기능에 대한 개요"
# 에이전트를 호출하고 응답을 반환합니다
response = research_agent(query)

이제 이 에이전트를 도구로 래핑할 수 있습니다. 다른 에이전트들이 이와 상호작용할 수 있게 됩니다.

#### 도구로서의 에이전트 모범 사례

Strands AI로 "도구로서의 에이전트" 패턴을 구현할 때:

1. 명확한 도구 문서화: 에이전트의 전문 분야를 설명하는 서술적 docstring 작성
2. 집중된 시스템 프롬프트: 각 전문 에이전트를 해당 도메인에 집중하도록 유지
3. 적절한 응답 처리: 응답을 추출하고 형식화하는 일관된 패턴 사용
4. 도구 선택 가이드: 오케스트레이터에게 각 전문 에이전트를 언제 사용할지에 대한 명확한 기준 제공

In [ ]:
@tool
def research_assistant(query: str) -> str:
    """
    연구 관련 쿼리를 처리하고 응답합니다.

    Args:
        query: 사실적 정보가 필요한 연구 질문

    Returns:
        인용이 포함된 상세한 연구 답변
    """
    try:
        # Strands agents를 사용하면 전문 에이전트를 쉽게 만들 수 있습니다
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # 에이전트를 호출하고 응답을 반환합니다
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"연구 어시스턴트 오류: {str(e)}"

이제 모범 사례를 따라 `product_recommendation_assistant`, `trip_planning_assistant`, 그리고 `orchestrator` 에이전트를 만들어보겠습니다.

### 제품 추천 어시스턴트

In [ ]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    적절한 제품을 제안하여 제품 추천 쿼리를 처리합니다.

    Args:
        query: 사용자 선호도가 포함된 제품 문의

    Returns:
        이유가 포함된 개인화된 제품 추천
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
            system_prompt="""당신은 전문 제품 추천 어시스턴트입니다.
            사용자 선호도를 기반으로 개인화된 제품 제안을 제공하세요. 항상 출처를 인용하세요.""",
        )
        # 에이전트를 호출하고 응답을 반환합니다
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"제품 추천 오류: {str(e)}"

In [ ]:
product_recommendation_assistant("플라잉카에 대한 제품 추천")

### 여행 계획 어시스턴트

In [ ]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    여행 일정을 만들고 여행 조언을 제공합니다.

    Args:
        query: 목적지와 선호도가 포함된 여행 계획 요청

    Returns:
        상세한 여행 일정 또는 여행 조언
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
            system_prompt="""당신은 전문 여행 계획 어시스턴트입니다.
            사용자 선호도를 기반으로 상세한 여행 일정을 만드세요.""",
        )
        # 에이전트를 호출하고 응답을 반환합니다
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"여행 계획 오류: {str(e)}"

### 오케스트레이터 에이전트

In [ ]:
# 명확한 도구 선택 가이드가 포함된 오케스트레이터 시스템 프롬프트 정의
MAIN_SYSTEM_PROMPT = """
당신은 쿼리를 전문 에이전트로 라우팅하는 어시스턴트입니다:
- 연구 질문과 사실적 정보 → research_assistant 도구 사용
- 제품 추천과 쇼핑 조언 → product_recommendation_assistant 도구 사용
- 여행 계획과 일정 → trip_planning_assistant 도구 사용
- 전문 지식이 필요하지 않은 간단한 질문 → 직접 답변

항상 사용자의 쿼리를 기반으로 가장 적절한 도구를 선택하세요.
"""

In [ ]:
# Strands Agents는 에이전트 도구의 쉬운 통합을 허용합니다
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [ ]:
# 예시: 전자상거래 고객 서비스 시스템
customer_query = (
    "등산화를 찾고 있습니다. 최종 응답을 현재 디렉토리에 작성해주세요."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# 오케스트레이터가 자동으로 여러 전문 에이전트가 필요하다고 판단합니다
response = orchestrator(customer_query)

오케스트레이터의 메시지를 살펴보겠습니다. 여기서 에이전트가 하위 에이전트를 도구로 사용하기로 결정한 것을 볼 수 있습니다

In [ ]:
orchestrator.messages

In [ ]:
customer_query = "파타고니아 여행 계획을 도와주실 수 있나요?"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

### 여러 에이전트 호출하기

In [ ]:
orchestrator.messages = []

In [ ]:
query = "스페인에 대한 연구를 해주실 수 있나요? 또한 7일 여행 계획도 도와주세요."

orchestrator(query)

내부적으로 오케스트레이터는 다음과 같이 작동합니다:
1. 먼저 `research_assistant`를 호출합니다
2. 그다음 `trip_planning_assistant`를 호출합니다
3. 이러한 전문 응답들을 두 쿼리 모두를 다루는 일관된 답변으로 결합합니다

### 순차적 에이전트 통신 패턴


에이전트 도구는 여러 에이전트를 함께 결합할 수도 있습니다. 이 예제에서는 `research_agent`의 출력을 `summary_agent`에 제공하고 요약된 응답을 반환하겠습니다.

In [ ]:
 # 사용자 쿼리 정의
topic = "생성형 AI"
# 연구 에이전트 생성
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# 요약 에이전트 생성
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    당신은 복잡한 정보를 명확하고 간결한 요약으로 정제하는 데 집중하는 요약 전문가입니다.
    주요 목표는 상세한 정보를 받아 핵심 포인트, 주요 논점, 중요한 데이터를 추출하는 것입니다.
    원본 내용의 정확성을 유지하면서 더 소화하기 쉽게 만들어야 합니다.
    명확성, 간결성, 그리고 정보의 가장 중요한 측면을 강조하는 데 집중하세요.
    """,
)

print("여러 에이전트가 성공적으로 생성되었습니다!")
print(f"\n🔍 연구 에이전트가 작업 중: {topic}\n") 
try:
    # 에이전트 1: 연구 에이전트 호출
    research_response = research_agent(
        f"{topic}에 대한 포괄적인 정보를 수집해주세요."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ 요약 에이전트가 연구를 정제하는 중\n")
    
    # 에이전트 2: 요약 에이전트에게 간결한 요약 생성 요청
    summary_response = summary_agent(
        f"이 연구의 간결한 요약을 만들어주세요: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"연구 어시스턴트 오류: {str(e)}")

## 축하합니다!

Strands Agents에서 에이전트를 도구로 사용하여 더 복잡한 에이전트 애플리케이션을 만드는 방법을 배웠습니다